In [27]:
import numpy as np
import tensorflow as tf
import numpy as np
from music21 import stream, instrument, note, chord
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras.models import *


try:
  import google.colab
  IS_ON_GOOGLE_COLAB = True
except:
  IS_ON_GOOGLE_COLAB = False

if IS_ON_GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [28]:
import glob
import os
import pickle

from music21 import converter, pitch, interval, instrument, note, chord
import tensorflow as tf
# Define save directory
from music21.key import Key
import numpy as np
from pathlib import Path

midi_dir = './midi_songs/'


def get_current_datetime():
    from datetime import datetime
    now = datetime.now()
    dt_name = now.strftime("%m_%d_%Y__%H_%M_%S")
    return dt_name

if IS_ON_GOOGLE_COLAB:
  FOLDER_ROOT = "/content/drive/MyDrive/magisterka/SheetMusicGenerator2"
else:
  FOLDER_ROOT = "."
TEST_RUN = True
NORMALIZE_NOTES = True
USE_COMPUTED_VALUES = False
USE_SAVE_POINT = False

NORMALIZATION_BOUNDARIES = [3, 4]
EPOCHS = 250
LATENT_VECTOR_DIM = 2
BATCH_SIZE = 256
SEQUENCE_LENGTH = 32
# COMPUTED_INT_TO_NOTE_PATH = "/content/drive/MyDrive/magisterka/SheetMusicGenerator2/AUTOENCODER/data/dicts/int_to_note_08_19_2021__17_25_44"
# COMPUTED_INT_TO_DURATION_PATH = "/content/drive/MyDrive/magisterka/SheetMusicGenerator2/AUTOENCODER/data/dicts/int_to_duration_08_19_2021__17_25_44"
# COMPUTED_NOTES_PATH = "/content/drive/MyDrive/magisterka/SheetMusicGenerator2/AUTOENCODER/data/notes/notes_08_19_2021__17_25_44"
# COMPUTED_DURATIONS_PATH = "/content/drive/MyDrive/magisterka/SheetMusicGenerator2/AUTOENCODER/data/durations/durations_08_19_2021__17_25_44"
COMPUTED_DATA_PATH = "AUTOENCODER/data/data_file_12_02_2021__18_08_03"



SAVE_POINT = "AUTOENCODER/checkpoints/08_19_2021__18_34_10/epoch=014-loss=383.5284-acc=0.0000.hdf5"
AUTOENCODER = "AUTOENCODER"

MODEL_NAME = AUTOENCODER
MODEL_FOLDER_ROOT = os.path.join(FOLDER_ROOT, MODEL_NAME)
CURR_DT = get_current_datetime()
MODEL_DIR_PATH = os.path.join(MODEL_FOLDER_ROOT, "generated_models")
OCCURENCES = os.path.join(MODEL_FOLDER_ROOT, "data", "occurences")

DATA_DIR = os.path.join(MODEL_FOLDER_ROOT, "data")
DATA_NOTES_DIR = os.path.join(DATA_DIR, "notes")
DATA_DURATIONS_DIR = os.path.join(DATA_DIR, "durations")

DATA_FILE_PATH = os.path.join(DATA_DIR, "data_file_" + str(CURR_DT))

DATA_DICTS_DIR = os.path.join(DATA_DIR, "dicts")
DATA_INT_TO_NOTE_PATH = os.path.join(DATA_DICTS_DIR, "int_to_note_" + str(CURR_DT))
DATA_INT_TO_DURATION_PATH = os.path.join(DATA_DICTS_DIR, "int_to_duration_" + str(CURR_DT))
DATA_NOTES_PATH = os.path.join(DATA_NOTES_DIR, "notes_" + str(CURR_DT))

DATA_DURATIONS_PATH = os.path.join(DATA_DURATIONS_DIR, "durations_" + str(CURR_DT))
# MIDI_SONGS_DIR = os.path.join(FOLDER_ROOT, "midi_songs")
MIDI_SONGS_DIR = os.path.join(FOLDER_ROOT, "midi_songs_smaller")
# MIDI_SONGS_DIR = os.path.join(FOLDER_ROOT, "midi_songs_medium")
MIDI_GENERATED_DIR = os.path.join(MODEL_FOLDER_ROOT, "midi_generated")
MIDI_SONGS_REGEX = os.path.join(MIDI_SONGS_DIR, "*.mid")
CHECKPOINTS_DIR = os.path.join(MODEL_FOLDER_ROOT, "checkpoints")
CHECKPOINT = os.path.join(CHECKPOINTS_DIR, str(CURR_DT))
LOGS_DIR = os.path.join(MODEL_FOLDER_ROOT, "logs")

LOG = os.path.join(LOGS_DIR, str(CURR_DT))
all_paths = [MODEL_DIR_PATH, OCCURENCES, DATA_NOTES_DIR, DATA_DURATIONS_DIR, DATA_DICTS_DIR,
             MIDI_GENERATED_DIR, CHECKPOINTS_DIR, CHECKPOINT, LOGS_DIR, LOG]

for path in all_paths:
    Path(path).mkdir(parents=True, exist_ok=True)

# def create_train_data():
# # Create empty list for scores
#     originalScores = []
#
#     # Load and make list of stream objects
#     for indx, song in enumerate(glob.glob(MIDI_SONGS_REGEX)):
#
#         print("Parsing song: " + str(song))
#         score = converter.parse(song)
#         originalScores.append(score)
#         if TEST_RUN and indx == 1:
#             break
#
#     # Define empty lists of lists
#     originalChords = [[] for _ in originalScores]
#     originalDurations = [[] for _ in originalScores]
#     originalKeys = []
#
#     def transpose_amount(score):
#         return -int(score.chordify().analyze('key').tonic.ps % 12)
#
#     def monophonic(stream):
#         try:
#             length = len(instrument.partitionByInstrument(stream).parts)
#         except:
#             length = 0
#         return length == 1
#     # Extract notes, chords, durations, and keys
#
#
#     originalScores = [song.chordify() for song in originalScores]
#
#     for i, song in enumerate(originalScores):
#         originalKeys.append(str(song.analyze('key')))
#         # song.transpose
#         transp_int = transpose_amount(song)
#         for element in song:
#             if isinstance(element, note.Note):
#                 originalChords[i].append(element.pitch.transpose(transp_int))
#                 originalDurations[i].append(element.duration.quarterLength)
#             elif isinstance(element, chord.Chord):
#                 originalChords[i].append('.'.join(str(n.transpose(transp_int)) for n in element.pitches))
#                 originalDurations[i].append(element.duration.quarterLength)
#         print(str(i))
#
#     cChords = [c for (c, k) in zip(originalChords, originalKeys) if (k == 'C major')]
#     cDurations = [c for (c, k) in zip(originalDurations, originalKeys) if (k == 'C major')]
#     # Map unique chords to integers
#     uniqueChords = np.unique([i for s in originalChords for i in s])
#     chordToInt = dict(zip(uniqueChords, list(range(0, len(uniqueChords)))))
#
#     # Map unique durations to integers
#     uniqueDurations = np.unique([i for s in originalDurations for i in s])
#     durationToInt = dict(zip(uniqueDurations, list(range(0, len(uniqueDurations)))))
#
#     # Print number of unique notes and chords
#     print(len(uniqueChords))
#
#     # Print number of unique durations
#     print(len(uniqueDurations))
#
#     intToChord = {i: c for c, i in chordToInt.items()}
#     intToDuration = {i: c for c, i in durationToInt.items()}
#
#     # Define sequence length
#     sequenceLength = 32
#
#     # Define empty arrays for train data
#     trainChords = []
#     trainDurations = []
#
#     # Construct training sequences for chords and durations
#     for s in range(len(cChords)):
#         chordList = [chordToInt[c] for c in cChords[s]]
#         durationList = [durationToInt[d] for d in cDurations[s]]
#         for i in range(len(chordList) - sequenceLength):
#             trainChords.append(chordList[i:i+sequenceLength])
#             trainDurations.append(durationList[i:i+sequenceLength])
#
#     with open(DATA_NOTES_DIR + "/notes", 'wb') as filepath:
#         pickle.dump(cChords, filepath)
#
#     with open(DATA_DURATIONS_DIR + "/durations", 'wb') as filepath:
#         pickle.dump(cDurations, filepath)
# #
# #     trainChords = tf.keras.utils.to_categorical(trainChords).transpose(0,2,1)
# #
# # # Convert data to numpy array of type float
# #     trainChords = np.array(trainChords, np.float)
#
# # Flatten sequence of chords into single dimension
#         # Convert to one-hot encoding and swap chord and sequence dimensions
#     trainChords = tf.keras.utils.to_categorical(trainChords).transpose(0, 2, 1)
#
#     # Convert data to numpy array of type float
#     trainChords = np.array(trainChords, np.float)
#
#     nSamples = trainChords.shape[0]
#     nChords = trainChords.shape[1]
#     inputDim = nChords * sequenceLength
#     # Flatten sequence of chords into single dimension
#     trainChordsFlat = trainChords.reshape(nSamples, inputDim)
#
#     return trainChordsFlat, inputDim, trainDurations, sequenceLength, intToChord, intToDuration, nChords
# if __name__ == "__main__":
#     create_train_data()
# # Convert to one-hot encoding and swap chord and sequence dimensions

In [29]:
class Trainer():
    def __init__(self, model, x, y):
        self.model = model
        self.x = x
        self.y = y
        self.steps_per_epoch = len(x)

    def train(self, checkpoint_path=None):
        # Define number of samples, chords and notes, and input dimension
        # filepath = CHECKPOINTS + "weights-improvement-{epoch:02d}-{loss:.4f}-{categorical_accuracy:.4f}-bigger.hdf5"
        # filepath = "weights-improvement-epoch:{epoch:02d}-loss:{loss:.4f}-cat_acc:{categorical_accuracy:.4f}.hdf5"
        if checkpoint_path:
            self.model.load_weights(checkpoint_path)
            nb_epoch = int(os.path.basename(checkpoint_path).split("=")[1].split("-")[0])

        else:
            nb_epoch = 0
        filepath = os.path.join(CHECKPOINT, "epoch={epoch:03d}-loss={loss:.4f}-acc={binary_crossentropy:.4f}.hdf5")

        # filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
        checkpoint = ModelCheckpoint(
            filepath,
            monitor='loss',
            verbose=0,
            save_best_only=True,
            mode='min'
        )
        log = tf.keras.callbacks.TensorBoard(log_dir=LOG + " " + str(self.model))


        callbacks_list = [checkpoint, log]
        # history = self.model.fit(network_input, network_output, epochs=EPOCHS, batch_size=128, callbacks=callbacks_list)
        # model.save(MODEL_DIR_PATH + MODEL_NAME + "_" + CURR_DT + ".hdf5")

        self.model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001),
                           metrics=[tf.keras.losses.BinaryCrossentropy()])
        # self.model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
        # Train autoencoder
        self.model.summary()
        print(MODEL_DIR_PATH + MODEL_NAME + "_" + CURR_DT + ".hdf5")
        # history = self.model.fit(self.trainNotesFlat, self.trainNotesFlat, epochs=1)
        # history = self.model.fit(self.trainNotesFlat, self.trainNotesFlat, epochs=500, callbacks=callbacks_list, batch_size=8)
        # tensor_dataset = tf.data.Dataset.from_tensors((self.trainNotesFlat, self.trainNotesFlat))

        # history = self.model.fit(x=self.data_generator(),
        #                          epochs=EPOCHS,
        #                          callbacks=callbacks_list,
        #                          batch_size=BATCH_SIZE,
        #                          steps_per_epoch=self.steps_per_epoch)
        # history = self.model.fit_generator(self.data_generator(),
        #                          epochs=EPOCHS,
        #                          callbacks=callbacks_list,
        #                          initial_epoch=nb_epoch,
        #                          # batch_size=BATCH_SIZE,
        #                          steps_per_epoch=self.steps_per_epoch)

        history = self.model.fit(x=self.x,
                                 y=self.y,
                         epochs=EPOCHS,
                         callbacks=callbacks_list,
                         initial_epoch=nb_epoch,
                         # batch_size=BATCH_SIZE,
                         steps_per_epoch=self.steps_per_epoch)
        print(history.history)
        print(MODEL_DIR_PATH + MODEL_NAME + "_" + CURR_DT + ".hdf5")
        self.model.save(os.path.join(MODEL_DIR_PATH, MODEL_NAME + "_" + CURR_DT + ".hdf5"))

In [34]:
class PreparedData:
    def __init__(self):
        self.train_notes = None
        self.train_durations = None
        self.int_to_note = None
        self.int_to_duration = None
        self.sequence_length = None

        self.n_of_unique_notes_classes = None
        self.input_dim = None
        self.dataset = None

    def load_prepared_data(self):
        with open(COMPUTED_DATA_PATH, 'rb') as data_file:
            self.train_notes, self.train_durations, self.int_to_note, self.int_to_duration, self.sequence_length = pickle.load(data_file)

    def save_values(self):
        with open(DATA_FILE_PATH, 'wb') as filepath:
            pickle.dump([self.train_notes, self.train_durations, self.int_to_note, self.int_to_duration, self.sequence_length], filepath)

        # with open(DATA_DURATIONS_PATH, 'rb') as train_durations_file:
        #     self.train_durations = pickle.load(train_durations_file)
        #
        # with open(DATA_INT_TO_NOTE_PATH, 'rb') as int_to_note_file:
        #     self.int_to_note = pickle.load(int_to_note_file)
        #
        # with open(DATA_INT_TO_DURATION_PATH, 'rb') as int_to_duration_file:
        #     self.int_to_duration = pickle.load(int_to_duration_file)

    def parse_songs(self, sequence_length):
        self.sequence_length = sequence_length
        # Create empty list for scores
        original_scores = []

        # Load and make list of stream objects
        for indx, song in enumerate(glob.glob(MIDI_SONGS_REGEX)):
            print("Parsing song: " + str(song))
            score = converter.parse(song)
            original_scores.append(score)
            if TEST_RUN and indx == 1:
                break

        # Define empty lists of lists
        original_notes = [[] for _ in original_scores]
        original_durations = [[] for _ in original_scores]
        original_keys = []

        def transpose_amount(score):
            return -int(score.chordify().analyze('key').tonic.ps % 12)

        def monophonic(stream):
            try:
                length = len(instrument.partitionByInstrument(stream).parts)

            except:
                length = 0
            return length == 1

        # Extract notes, notes, durations, and keys

        original_scores = [song.chordify() for song in original_scores]

        for i, song in enumerate(original_scores):
            transp_int = transpose_amount(song)
            original_keys.append(str(song.analyze('key').transpose(transp_int)))
            for element in song:
                if isinstance(element, note.Note):
                    original_notes[i].append(element.pitch.transpose(transp_int))
                    original_durations[i].append(element.duration.quarterLength)

                elif isinstance(element, chord.Chord):
                    original_notes[i].append('.'.join(str(n.transpose(transp_int)) for n in element.pitches))
                    original_durations[i].append(element.duration.quarterLength)
            print(str(original_keys[i]))

        c_notes = [c for (c, k) in zip(original_notes, original_keys) if (k == 'C major')]
        c_durations = [c for (c, k) in zip(original_durations, original_keys) if (k == 'C major')]
        # Map unique notes to integers
        unique_notes = np.unique([i for s in original_notes for i in s])
        note_to_int = dict(zip(unique_notes, list(range(0, len(unique_notes)))))

        # Map unique durations to integers
        unique_durations = np.unique([i for s in original_durations for i in s])
        duration_to_int = dict(zip(unique_durations, list(range(0, len(unique_durations)))))

        # Print number of unique notes and notes
        print(len(unique_notes))

        # Print number of unique durations
        print(len(unique_durations))

        int_to_note = {i: c for c, i in note_to_int.items()}
        int_to_duration = {i: c for c, i in duration_to_int.items()}

        # Define sequence length

        # Define empty arrays for train data
        train_notes = []
        train_durations = []

        # Construct training sequences for notes and durations
        for s in range(len(c_notes)):
            note_list = [note_to_int[c] for c in c_notes[s]]
            duration_list = [duration_to_int[d] for d in c_durations[s]]
            for i in range(len(note_list) - sequence_length):
                train_notes.append(note_list[i:i + sequence_length])
                train_durations.append(duration_list[i:i + sequence_length])

        self.train_notes = np.array(train_notes)
        self.train_durations = np.array(train_durations)
        self.int_to_note = np.array(int_to_note)
        self.int_to_duration = np.array(int_to_duration)

        # return train_notes, train_durations, int_to_note, int_to_duration

        # with open(DATA_NOTES_PATH, 'wb') as filepath:
        #     pickle.dump(self.train_notes, filepath)
        #
        # with open(DATA_DURATIONS_PATH, 'wb') as filepath:
        #     pickle.dump(self.train_durations, filepath)
        #
        # with open(DATA_INT_TO_NOTE_PATH, 'wb') as filepath:
        #     pickle.dump(self.int_to_note, filepath)
        #
        # with open(DATA_INT_TO_DURATION_PATH, 'wb') as filepath:
        #     pickle.dump(self.int_to_duration, filepath)
        # self.train_notes, = train_notes
        # self.train_durations, = train_durations
        # self.int_to_note, = int_to_note
        # self.int_to_duration, = int_to_duration

    # @staticmethod
    def prepare_data(self, sequence_length):
        # print("trainNotesFlat: " + str(train_notes))

        # train_notes_categorical_not_transposed = tf.keras.utils.to_categorical(self.train_notes, dtype="float16")
        # Convert data to numpy array of type float
        # trainNotes = np.array(trainNotes, np.float32)
        train_notes_categorical = tf.keras.utils.to_categorical(self.train_notes, dtype="float16")  # .transpose(0, 2, 1)

        # print(f"train_notes_categorical.device: {train_notes_categorical.device}")
        n_samples = train_notes_categorical.shape[0]
        self.n_of_unique_notes_classes = train_notes_categorical.shape[2]
        self.input_dim = self.n_of_unique_notes_classes * sequence_length
        # Flatten sequence of notes into single dimension
        # train_notes_flattened = train_notes_categorical.reshape(n_samples, self.input_dim)
        self.dataset = train_notes_categorical.reshape(-1, self.input_dim)
        print("type(train_notes_flattened): " + str(type(self.dataset)))
        # self.numpy_dataset = train_notes_flattened
        return self.input_dim, self.dataset, self.n_of_unique_notes_classes


class MusicAutoencoder(tf.keras.Model):
    def __init__(self, latent_dim):
        super(MusicAutoencoder, self).__init__()
        # self.prepared_data = prepared_data
        self.latent_dim = latent_dim
        # self.sequence_length = sequence_length
        self.tensor_dataset = None
        self.input_dim = None
        self.n_of_unique_notes_classes = None
        self.numpy_dataset = None
        # self.train_notes = prepared_data.train_notes
        # self.train_notes_path = train_notes_path
        # self.train_durations_path = train_durations_path
        # self.int_to_note_path = int_to_note_path
        # self.int_to_duration_path = int_to_duration_path

        self.encoder = tf.keras.Sequential([
                Input(shape=SEQUENCE_LENGTH, batch_size=BATCH_SIZE),
                Dense(self.latent_dim, activation='tanh')
            ])

        self.decoder = tf.keras.Sequential([
                Input(shape=self.latent_dim),
                Dense(SEQUENCE_LENGTH, activation='sigmoid')
            ])
        # self.encoder = tf.keras.Sequential([
        #     tf.keras.layers.Input(shape = input_dim),
        #     tf.keras.layers.Dense(input_dim, activation = 'tanh')
        # ])
        #
        # self.decoder = tf.keras.Sequential([
        #     tf.keras.layers.Input(shape =input_dim),
        #     tf.keras.layers.Dense(input_dim, activation = 'sigmoid')
        # ])


    def call(self, inputs):
        # encoderInput = tf.keras.layers.Input(shape = self.inputDim)
        # latent = tf.keras.layers.Input(shape = self.latentDim)
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

    # def autoencoder(self, inputDim, latentDim):
    #     # Define encoder input shape
    #     encoderInput = tf.keras.layers.Input(shape =inputDim)
    #
    #     # Define decoder input shape
    #     latent = tf.keras.layers.Input(shape =latentDim)
    #
    #     # Define dense encoding layer connecting input to latent vector
    #     encoded = tf.keras.layers.Dense(latentDim, activation = 'tanh')(encoderInput)
    #
    #     # Define dense decoding layer connecting latent vector to output
    #     decoded = tf.keras.layers.Dense(inputDim, activation = 'sigmoid')(latent)
    #
    #     # Define the encoder and decoder models
    #     self.encoder = tf.keras.Model(encoderInput, encoded)
    #     self.decoder = tf.keras.Model(latent, decoded)
    #
    #     # Define autoencoder model
    #     autoencoder = tf.keras.Model(encoderInput, self.decoder(encoded))
    #     return autoencoder



    def train(self, prepared_data, checkpoint_path=None):
        # Define number of samples, chords and notes, and input dimension
        # filepath = CHECKPOINTS + "weights-improvement-{epoch:02d}-{loss:.4f}-{categorical_accuracy:.4f}-bigger.hdf5"
        # filepath = "weights-improvement-epoch:{epoch:02d}-loss:{loss:.4f}-cat_acc:{categorical_accuracy:.4f}.hdf5"
        curr_dt = get_current_datetime()
        print(str("Current datatime: " + curr_dt))

        if checkpoint_path:
            self.model.load_weights(checkpoint_path)

        filepath = CHECKPOINT + str(curr_dt) + "/" + "epoch:{epoch:02d}-loss:{loss:.4f}-acc:{binary_accuracy:.4f}.hdf5"

        # filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
        checkpoint = ModelCheckpoint(
            filepath,
            monitor='binary_accuracy',
            verbose=0,
            save_best_only=True,
            mode='max'
        )
        log = tf.keras.callbacks.TensorBoard(log_dir=LOG + curr_dt),

        callbacks_list = [checkpoint, log]
        # history = self.model.fit(network_input, network_output, epochs=EPOCHS, batch_size=128, callbacks=callbacks_list)
        # model.save(MODEL_DIR_PATH + MODEL_NAME + "_" + curr_dt + ".hdf5")

        self.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=[tf.keras.metrics.BinaryAccuracy()])
        # self.summary()
        # self.model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
        # Train autoencoder

        print(MODEL_DIR_PATH + MODEL_NAME + "_" + curr_dt + ".hdf5")
        # history = self.model.fit(self.trainChordsFlat, self.trainChordsFlat, epochs=1)
        # history = self.model.fit(self.trainChordsFlat, self.trainChordsFlat, epochs=500, callbacks=callbacks_list)
        print(str(prepared_data.dataset.shape))
        # print(str(self.prepared_data.shape))
        history = self.fit(prepared_data.dataset, prepared_data.dataset, epochs=EPOCHS, callbacks=callbacks_list)
        print(history.history)
        print(MODEL_DIR_PATH + MODEL_NAME + "_" + curr_dt + ".hdf5")
        self.save(MODEL_DIR_PATH + MODEL_NAME + "_" + curr_dt + ".hdf5")


    def generate_notes(self):
        generated_notes = self.decoder(np.random.normal(size=(1, self.latent_dim)))\
            .numpy().reshape(self.n_of_unique_notes_classes, self.sequence_length)\
            .argmax(0)

        generated_stream = stream.Stream()
        generated_stream.append(instrument.Piano())
        note_sequence = [self.int_to_note[c] for c in generated_notes]
        # Append notes and notes to stream object
        for j in range(len(note_sequence)):
            try:
                generated_stream.append(note.Note(note_sequence[j].replace('.', ' ')))
            except:
                generated_stream.append(note.Note(note_sequence[j].replace('.', ' ')))

        generated_stream.write('midi', fp=MIDI_GENERATED_DIR + 'autoencoder.mid')



class ModelFactory:
    @staticmethod
    def create_model(model_type):
        if model_type == AUTOENCODER:
            model = MusicAutoencoder(LATENT_VECTOR_DIM)
            return model

    @staticmethod
    def create_prepared_data(use_computed_values):
        prepared_data = PreparedData()
        if use_computed_values:
            prepared_data.load_prepared_data()
        else:
            prepared_data.parse_songs(SEQUENCE_LENGTH)
            prepared_data.save_values()

        prepared_data.prepare_data(SEQUENCE_LENGTH)
        return prepared_data


In [36]:
# modelFactory = ModelFactory()
prepared_data = ModelFactory.create_prepared_data(USE_COMPUTED_VALUES)
musicAutoEncoder = ModelFactory.create_model(AUTOENCODER)
musicAutoEncoder.train(prepared_data)
# musicAutoEncoder.summary()
musicAutoEncoder.generateChords()

Parsing song: ./midi_songs_smaller/caitsith.mid
Parsing song: ./midi_songs_smaller/decisive.mid
C major
c minor
374
10
type(train_notes_flattened): <class 'numpy.ndarray'>
Current datatime: 12_06_2021__16_22_52


2021-12-06 16:22:52.333317: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-06 16:22:52.333367: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-06 16:22:52.333741: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


./AUTOENCODER/generated_modelsAUTOENCODER_12_06_2021__16_22_52.hdf5
(888, 11936)
Epoch 1/250


ValueError: in user code:

    /home/nexon/OneDrive/Studia/Praca_magisterska/Projekt/SheetMusicGenerator2/venv/lib/python3.7/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /tmp/ipykernel_18496/3540579809.py:192 call  *
        encoded = self.encoder(inputs)
    /home/nexon/OneDrive/Studia/Praca_magisterska/Projekt/SheetMusicGenerator2/venv/lib/python3.7/site-packages/keras/engine/base_layer.py:1020 __call__  **
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    /home/nexon/OneDrive/Studia/Praca_magisterska/Projekt/SheetMusicGenerator2/venv/lib/python3.7/site-packages/keras/engine/input_spec.py:254 assert_input_compatibility
        ' but received input with shape ' + display_shape(x.shape))

    ValueError: Input 0 of layer sequential_6 is incompatible with the layer: expected axis -1 of input shape to have value 32 but received input with shape (None, 11936)
